In [1]:
import sys

sys.path.append("../")

import pandas as pd
import os
import matplotlib.pyplot as plt
from task1.retrieval_system import RetrievalSystem, SongInfo
from task1.similarity_measure import (
    cosine_similarity,
    dot_product,
    manhattan_distance,
    euclidean_distance,
    random_similarity,
)

from utils import embed_and_merge

In [2]:
def read(feature, h=0):
    file_path = os.path.join(
        "..", "data", f"id_{feature}_mmsr.tsv"
    )
    return pd.read_csv(file_path, delimiter="\t", header=h)
df = read("information", 0)
# add genre information for metric calculation
genres = read("genres", 0)
# convert genre to actual list via eval
genres["genre"] = genres["genre"].apply(eval).apply(set)
df = df.merge(genres, on="id", how="left")
for text_feature in ["lyrics_tf-idf"]:
        stats = read(text_feature, 0)
        df = embed_and_merge(df, stats, text_feature.split("_")[1])
        
visual_feature = "resnet"
stats = read(visual_feature, 0)
df = embed_and_merge(df, stats, visual_feature)

for audio_feature in ["ivec256", "musicnn"]:
    stats = read(audio_feature, 0)
    df = embed_and_merge(df, stats, audio_feature)
    
# data for task 2 does not include the item with id "03Oc9WeMEmyLLQbj" = row 5
df = df.drop(5)
df = df.reset_index()
df.head()
df["resnet"].values[1].shape

(4096,)

In [3]:
res = pd.DataFrame(columns=["id","tfidf", "resnet", "ivec256"])
max_rows = 10
for row in df.itertuples():
    songInfo = SongInfo(title=row.song, artist=row.artist)
    rs_tfidf = RetrievalSystem(
        df=df,
        sim_metric=cosine_similarity,
        sim_feature="ivec256",
    )
    rs_resnet = RetrievalSystem(
        df=df,
        sim_metric=cosine_similarity,
        sim_feature="resnet",
    )
    rs_ivec256 = RetrievalSystem(
        df=df,
        sim_metric=cosine_similarity,
        sim_feature="ivec256",
    )
    res.loc[len(res)] = [row.id, ';'.join(rs_tfidf.retrieve(songInfo, 10)['id']), ';'.join(rs_resnet.retrieve(songInfo, 10)['id']), ';'.join(rs_ivec256.retrieve(songInfo, 10)['id'])]
    
res.to_csv("cached_results.csv", index=False)
    
  

KeyboardInterrupt: 